In [1]:
import bs4 as bs
import requests
import pickle
import pandas_datareader.data as web
import os
import datetime as dt
from collections import Counter
from sklearn import svm
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [2]:
cd dfs-stocks

C:\Users\vikasbhadoria\Desktop\ML_Project1\Python_for_finance\dfs-stocks


### Using requests to get the data from web and the using beautifulsoup to make it more readable

In [3]:
def sp500_companies():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find("table",{"class":"wikitable sortable"})
    tickers = []
    for row in table.findAll("tr")[1:20]:
        ticker = row.findAll("td")[0].text
        ticker = ticker[:-1]
        tickers.append(ticker)
    
    with open("sp20_companies.pickle","wb") as f:
        pickle.dump(tickers,f)
      
    print(tickers)
    
    return tickers
#sp500_companies()

### Getting the data from yahoo and storing it in csv format

In [4]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = sp500_companies()
    else:
        with open("sp20_companies.pickle","rb") as f:
            tickers = pickle.load(f)
            
    if not os.path.exists("dfs-stocks"):
        os.makedirs("dfs-stocks")
    
    start = dt.datetime(2000,1,1)
    end = dt.datetime(2016,12,31)
    
    for ticker in tickers[:]:
        if not os.path.exists("{}.csv".format(ticker)):
            df = web.DataReader(ticker,"yahoo",start,end)
            df.to_csv("{}.csv".format(ticker))
        else:
            print("Already have {}".format(ticker))
            
#get_data_from_yahoo()             


### Consider only Adj Close of all companies and then combining all the data into one csv

In [5]:
def compile_data():
    with open("sp20_companies.pickle","rb") as f:
        tickers = pickle.load(f)
        
    main_df = pd.DataFrame()
    
    for count,ticker in enumerate(tickers):
        df = pd.read_csv("{}.csv".format(ticker))
        df.set_index("Date",inplace=True)
        df.rename(columns = {"Adj Close":ticker},inplace=True)
        df.drop(["Open","High","Low","Close","Volume"],1,inplace=True)
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df,how="outer")
        
        if count % 2 == 0:
            print(count)
    print(main_df.head())
    main_df.to_csv("sp20_combined.csv")

#compile_data()

### Data Visualisation and corelation

In [6]:
def data_visualize():
    data = pd.read_csv("sp20_combined.csv")
    #print(data.head())
    corr_matrix = data.corr()
    plt.figure(figsize=(14,10))
    return sns.heatmap(corr_matrix,cmap="YlGnBu",linewidths=.5,annot=True)
data_visualize()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Creating labels for data based on weather to buy stock of a particular company or sell or hold. This depends on the %change in data from last 7 days

In [7]:
def process_data_for_labels(ticker):
    hm_days = 7
    data = pd.read_csv("sp20_combined.csv",index_col=0)
    tickers = data.columns.values.tolist()
    data.fillna(0,inplace=True)
    
    for i in range(1,hm_days+1):
        print(i)
        data["{}_{}d".format(ticker,i)] = (data[ticker].shift(-i)-data[ticker])/data[ticker]
    data.fillna(0,inplace=True)
    return tickers,data
#process_data_for_labels("AMD")

In [8]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > 0.018:
            return 1
        if col < -0.019:
            return -1
    return 0

In [9]:
def extract_features(ticker):
    tickers,data = process_data_for_labels(ticker)
    
    data["{}_target".format(ticker)] = list(map(buy_sell_hold,data["{}_1d".format(ticker)],
                                             data["{}_2d".format(ticker)],
                                             data["{}_3d".format(ticker)],
                                             data["{}_4d".format(ticker)],
                                             data["{}_5d".format(ticker)],
                                             data["{}_6d".format(ticker)],
                                             data["{}_7d".format(ticker)]))
    vals= data["{}_target".format(ticker)].values.tolist()
    str_values = [str(i) for i in vals]
    print("data_spread:" , Counter(str_values))
    data.fillna(0,inplace=True)
    data = data.replace([np.inf,-np.inf],np.nan)
    data.dropna(inplace=True)
    
    data_vals = data[[ticker for ticker in tickers]].pct_change()
    data_vals = data_vals.replace([np.inf,-np.inf],0)
    data_vals.fillna(0,inplace=True)
    
    X = data_vals.values
    y = data["{}_target".format(ticker)].values
    
    return X,y,data
#extract_features("ABT")
    

### Using Machine learning 

In [10]:
def do_ml(ticker):
    X,y,data = extract_features(ticker)
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
    
    clf = VotingClassifier([
        ("lsvc",svm.LinearSVC()),
        ("knn",KNeighborsClassifier()),
        ("rfor",RandomForestClassifier())])
    clf.fit(X_train,y_train)
    confidence = clf.score(X_test,y_test)
    print("Accuracy: ",confidence )
    predictions = clf.predict(X_test)
    print("prediction spread: ",Counter(predictions))
    
    return confidence

do_ml("MMM")

<IPython.core.display.Javascript object>

1
2
3
4
5
6
7
data_spread: Counter({'1': 1802, '-1': 1374, '0': 1101})


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Accuracy:  0.40373831775700936
prediction spread:  Counter({1: 726, -1: 220, 0: 124})


0.40373831775700936

### The End